In [1]:
import requests, json
import threading
import pandas as pd
import flask
from flask import Flask, request
import geopy
from geopy import Nominatim

ModuleNotFoundError: No module named 'requests'

In [ ]:
key = 'fsq3bR9nCSrR/WbzD82rlvh990Q70wuc8BuuRs0Ypm6fx+w='

# this method helps us get long and lat of certain city

def get_latlong(b):

    city = b

    geolocator = Nominatim(user_agent = 'Safra')

    loc = geolocator.geocode(city)

    return loc.latitude , loc.longitude

In [ ]:
lat , long = get_latlong('egypt')
lat , long

(26.2540493, 29.2675469)

In [ ]:
def retrieve_places(a , c):

    """
    a : condition --- >  example : coffee , art gallery , etc ...
    c : city name
    
    """

    lat , long = get_latlong(c)


    if a != '':
        fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&query={a}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation"

    else:
        fields_url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{long}&fields=fsq_id%2Cname%2Ctel%2Cprice%2Crating%2Cdescription%2Clocation"


    url = fields_url

    headers = {
        "Accept": "application/json",
        "Authorization": key
    }

    response = requests.get(url, headers=headers)

    data = response.json()

    df = pd.json_normalize(data['results'])

    #deleting unnecessary columns
    
    df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)


    # renaming columns


    df.rename(columns = {'location.country':'country' , 'location.region':'region'}, inplace = True)

    

    # filling nan values

    df = df.fillna('Not Available')



    # changing datatypes
    df = df.astype({'price': str , 'rating': str})

    data = df.to_json(orient = 'records')
    return df , data

In [ ]:
df, data_json = retrieve_places('' , 'Cairo')
df

C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)


,fsq_id,name,price,rating,country,region,tel
0,60539559afff524f93bef5cb,ستاربكس,1.0,8.5,EG,Muḩāfaz̧at al Qāhirah,Not Available
1,56e3232bcd1044008992e42c,كوستا كوفى,1.0,8.0,EG,Muḩāfaz̧at al Qāhirah,02 23955506
2,4c924268b641236a82568879,بيتزا هت,1.0,7.4,EG,Muḩāfaz̧at al Qāhirah,02 27941315
3,5ac227211f8ed65a8a965606,ستاربكس,1.0,8.5,EG,Muḩāfaz̧at al Jīzah,Not Available
4,541cbaf9498e3d028fdcc628,قهوة الكراسي البيضاء,1.0,7.7,EG,Muḩāfaz̧at al Qāhirah,Not Available
5,56be0a92498e1a2f8abeda87,Sikh Mashwi (سيخ مشوي),Not Available,8.2,EG,الجيزة,02 37605454
6,5e617321f04f090008ea176c,ستاربكس,1.0,9.3,EG,Muḩāfaz̧at al Qāhirah,Not Available
7,581b8fffbb5d611d3e42f3f6,Villa Garden City (الفيلا جاردن سيتي),1.0,7.4,EG,Muḩāfaz̧at al Qāhirah,Not Available
8,4bf2d2dae5eba59373d81d90,McDonald's (ماكدونالدز),1.0,8.1,EG,Muḩāfaz̧at al Jīzah,02 33372226
9,4c31702ea0ced13a76b7136e,تشيليز,Not Available,7.2,EG,Muḩāfaz̧at al Qāhirah,Not Available


In [ ]:


app = Flask(__name__)

@app.route('/api' , methods = ['GET'])

def index():
    userInputb = str(request.args['query'])
    df, data_json = retrieve_places('' , userInputb)

    return data_json




if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2022-09-24 01:55:20,435] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1519, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1517, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 1503, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\3309281160.py", line 7, in index
    df, data_json = retrieve_places('' , userInputb)
  File "C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py", line 8, in retrieve_places
    lat , lo

127.0.0.1 - - [24/Sep/2022 02:04:30] "GET /api?query= HTTP/1.1" 500 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [24/Sep/2022 02:04:33] "GET /api?query=R HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [24/Sep/2022 02:04:33] "GET /api?query=Ri HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future ve

127.0.0.1 - - [24/Sep/2022 02:04:36] "GET /api?query=Riyad HTTP/1.1" 500 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [24/Sep/2022 02:04:37] "GET /api?query=Riyadh HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
[2022-09-24 02:11:43,198] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\a

127.0.0.1 - - [24/Sep/2022 02:11:45] "GET /api?query=R HTTP/1.1" 500 -
[2022-09-24 02:11:45,099] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\geocoders\base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 438, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 466, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 404

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", 

...

127.0.0.1 - - [24/Sep/2022 02:12:05] "GET /api?query=du HTTP/1.1" 500 -
[2022-09-24 02:12:05,288] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\geocoders\base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 438, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 466, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 404

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py",

127.0.0.1 - - [24/Sep/2022 02:12:06] "GET /api?query=d HTTP/1.1" 500 -
[2022-09-24 02:12:06,174] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\geocoders\base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 438, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "C:\Users\Saud\miniconda3\lib\site-packages\geopy\adapters.py", line 466, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 404

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", line 2073, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Saud\miniconda3\lib\site-packages\flask\app.py", 

127.0.0.1 - - [24/Sep/2022 02:12:06] "GET /api?query= HTTP/1.1" 500 -
[2022-09-24 02:12:06,922] ERROR in app: Exception on /api [GET]
Traceback (most recent call last):
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "C:\Users\Saud\miniconda3\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 1377, in getresponse
    response.begin()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 320, in begin
    version, status, reason = self._read_status()
  File "C:\Users\Saud\miniconda3\lib\http\client.py", line 281, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
  File "C:\Users\Saud\miniconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\Saud\miniconda3\lib\

127.0.0.1 - - [24/Sep/2022 02:12:09] "GET /api?query=melo HTTP/1.1" 500 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [24/Sep/2022 02:12:09] "GET /api?query=melb HTTP/1.1" 200 -
C:\Users\Saud\AppData\Local\Temp\ipykernel_10212\276041794.py:33: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(df.columns.difference(['fsq_id', 'name', 'price', 'rating', 'tel', 'location.country', 'location.region', 'description']),1,inplace=True)
127.0.0.1 - - [24/Sep/2022 02:12:09] "GET /api?query=mel HTTP/1.1" 200 -
[2022-09-24 02:12:11,072] ERROR in app: Exception on /api [GET]
Traceback (most recent 

127.0.0.1 - - [24/Sep/2022 02:12:11] "GET /api?query=melborne HTTP/1.1" 500 -
